In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices())
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax

In [3]:
train_dir = r"C:\Users\thanawin\Desktop\Deep\detect\train" #passing the path with training images
test_dir = r"C:\Users\thanawin\Desktop\Deep\detect\test"   #passing the path with testing images

In [4]:
img_size = 128 #original size of the image

In [5]:
train_datagen = ImageDataGenerator(#rotation_range = 180,
                                         width_shift_range = 0.1,
                                         height_shift_range = 0.1,
                                         horizontal_flip = True,
                                         rescale = 1./255,
                                         #zoom_range = 0.2,
                                         validation_split = 0.2
                                        )
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)

In [6]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (img_size,img_size),
                                                    batch_size = 64,
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training"
                                                   )
validation_generator = validation_datagen.flow_from_directory( directory = test_dir,
                                                              target_size = (img_size,img_size),
                                                              batch_size = 64,
                                                              color_mode = "grayscale",
                                                              class_mode = "categorical",
                                                              subset = "validation"
                                                             )

Found 16364 images belonging to 7 classes.
Found 452 images belonging to 7 classes.


In [7]:
epochs = 60
batch_size = 64

In [8]:
model= tf.keras.models.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(128, 128,1)))
model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
    
model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) 
model.add(Dense(256,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
    
model.add(Dense(512,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

model.compile(
    optimizer = Adam(lr=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
  )

C:\Users\thanawin\.conda\envs\py2\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 128)       5

In [10]:
history = model.fit(x = train_generator,epochs = epochs,validation_data = validation_generator)

Epoch 1/60
256/256 [==============================] - 225s 772ms/step - loss: 8.9070 - accuracy: 0.3195 - val_loss: 9.9483 - val_accuracy: 0.1372
Epoch 2/60
256/256 [==============================] - 51s 199ms/step - loss: 7.6984 - accuracy: 0.4349 - val_loss: 9.7862 - val_accuracy: 0.1571
Epoch 3/60
256/256 [==============================] - 51s 200ms/step - loss: 6.6233 - accuracy: 0.4914 - val_loss: 9.5359 - val_accuracy: 0.1327
Epoch 4/60
256/256 [==============================] - 51s 200ms/step - loss: 5.6319 - accuracy: 0.5378 - val_loss: 8.3400 - val_accuracy: 0.2389
Epoch 5/60
256/256 [==============================] - 51s 200ms/step - loss: 4.7618 - accuracy: 0.5769 - val_loss: 5.6271 - val_accuracy: 0.272169 - ETA:  - ETA: 25s - loss: 4.9527 - a - ETA: 22s -  - ETA: 16s - loss: 4.8763 - accura - ETA: 14s - loss: 4.8657 - accuracy: 0 - ETA: 13s - loss: 4.8583 - a - ETA: 10s - loss: 4.8379 - ac
Epoch 6/60
256/256 [==============================] - 51s 200ms/step - loss: 4.0155 

256/256 [==============================] - 51s 200ms/step - loss: 0.7702 - accuracy: 0.8293 - val_loss: 1.4189 - val_accuracy: 0.6549ss: 0 - ETA: 1s - loss: 0.7
Epoch 32/60
256/256 [==============================] - 51s 200ms/step - loss: 0.7519 - accuracy: 0.8339 - val_loss: 1.5984 - val_accuracy: 0.59730.7494 - acc - ETA: 9s - loss: 0.748 - ETA: 7s - loss: 0.7503 - accuracy: 0.83 - ETA: 6s - loss: 0.7500 - accuracy: 0. - ETA: 6s - loss: -
Epoch 33/60
256/256 [==============================] - 51s 200ms/step - loss: 0.7553 - accuracy: 0.8301 - val_loss: 1.6061 - val_accuracy: 0.681445s - loss: 0.7541 - ac - ETA: 43s - loss: 0.7501 - ETA: 30s - loss: 0.7472 - accurac - ETA: 28s - loss: 0.7511 - accuracy: 0.83 - ETA: 28s - loss: 0.7515 - a - ETA: 25s - loss: 0.7527  - ETA: 2s - loss:
Epoch 34/60
256/256 [==============================] - 51s 200ms/step - loss: 0.7434 - accuracy: 0.8365 - val_loss: 1.7676 - val_accuracy: 0.5686ss: 0.7384 - accuracy: 0.8 - ETA: 24s - loss: 0.7382  - ETA: 

256/256 [==============================] - 51s 200ms/step - loss: 0.6644 - accuracy: 0.8737 - val_loss: 1.0042 - val_accuracy: 0.7611accu - ETA: 32s - loss: 0.6621 - ac - ETA: 21s - loss: 0.6635 - accuracy: 0 - ETA: 20s - loss: 0.6622 - acc - ETA: 17s - loss: 0.6604 - accuracy: 0.87 - ETA: 17s - loss: - ETA: 12s - loss: 0.
Epoch 59/60
256/256 [==============================] - 51s 200ms/step - loss: 0.6472 - accuracy: 0.8832 - val_loss: 1.3511 - val_accuracy: 0.72129 - accuracy - ETA: 32s - ETA: 26s - loss: 0.6398 - accuracy - ETA: 24s - loss: 0.6404  - ETA: 12s - loss: 0.6477  - ETA: 0s - loss: 0.6460 - accuracy
Epoch 60/60
256/256 [==============================] - 51s 200ms/step - loss: 0.6442 - accuracy: 0.8784 - val_loss: 1.3882 - val_accuracy: 0.7146
